https://nbodykit.readthedocs.io/en/latest/cookbook/convpower.html

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from nbodykit.lab import *
from nbodykit import setup_logging, style
#from nbodykit.source.catalog import FITSCatalog
#from nbodykit.algorithms import SurveyData2PCF

In [3]:
plt.style.use(style.notebook)
%matplotlib inline

In [4]:
setup_logging() # turn on logging to screen

In [5]:
# specify our cosmology
cosmo = cosmology.Cosmology(h=0.7).match(Omega0_m=0.31)
cosmo

In [6]:
datadir = os.path.join(os.getenv('IM_DATA_DIR'), 'sdss', 'dr12')

In [7]:
%time data = FITSCatalog(os.path.join(datadir, 'galaxy_DR12v5_CMASSLOWZTOT_North.fits.gz'))

[ 000003.65 ]   0: 12-18 06:11  CatalogSource   INFO     Extra arguments to FileType: ('/Users/ioannis/research/data/sdss/dr12/galaxy_DR12v5_CMASSLOWZTOT_North.fits.gz',)


CPU times: user 3.12 s, sys: 471 ms, total: 3.59 s
Wall time: 3.62 s


In [8]:
%time random = FITSCatalog(os.path.join(datadir, 'random0_DR12v5_CMASSLOWZTOT_North.fits.gz'))

[ 000053.89 ]   0: 12-18 06:12  CatalogSource   INFO     Extra arguments to FileType: ('/Users/ioannis/research/data/sdss/dr12/random0_DR12v5_CMASSLOWZTOT_North.fits.gz',)


CPU times: user 43.4 s, sys: 6.91 s, total: 50.3 s
Wall time: 50.2 s


In [9]:
# add Cartesian position column
data['Position'] = transform.SkyToCartesian(data['RA'], data['DEC'], data['Z'], cosmo=cosmo)
random['Position'] = transform.SkyToCartesian(random['RA'], random['DEC'], random['Z'], cosmo=cosmo)

In [10]:
random.columns

['AIRMASS',
 'DEC',
 'EB_MINUS_V',
 'IMAGE_DEPTH',
 'IPOLY',
 'ISECT',
 'NZ',
 'PSF_FWHM',
 'Position',
 'RA',
 'SKYFLUX',
 'Selection',
 'Value',
 'WEIGHT_FKP',
 'Weight',
 'Z',
 'ZINDX']

In [11]:
data.csize, random.csize

(953255, 48742266)

### Generate n(z) for the data.

In [12]:
4 * np.pi * (180 / np.pi)**2

41252.96124941928

In [13]:
# compute n(z) from the randoms
fsky = 9329 / ( 4 * np.pi * (180 / np.pi)**2 ) # sky fraction from Alam+17
%time zhist = RedshiftHistogram(random, fsky, cosmo, redshift='Z')

KeyboardInterrupt: 

In [14]:
# re-normalize to the total size of the data catalog
alpha = 1.0 * data.csize / random.csize

# add n(z) from randoms to the FKP source
nofz = InterpolatedUnivariateSpline(zhist.bin_centers, alpha*zhist.nbar)

NameError: name 'InterpolatedUnivariateSpline' is not defined

In [ ]:
plt.plot(zhist.bin_centers, alpha*zhist.nbar)
plt.xlabel(r'$z$', fontsize=16)
plt.ylabel(r'$n(z)$ $[h^{3} \mathrm{Mpc}^{-3}]$', fontsize=16)

In [ ]:
help(InterpolatedUnivariateSpline)

### Instantiate an FKPCatalog object by combining the data and randoms.

In [ ]:
fkp = FKPCatalog(data, random)

In [ ]:
fkp.columns

In [ ]:
# add the n(z) columns to the FKPCatalog
fkp['random/NZ'] = nofz(random['Z'])
fkp['data/NZ'] = nofz(data['Z'])